AQI PREDICTION SYSTEM

In [ ]:
pip install -U 'hopsworks[python]'

In [18]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import hopsworks
import joblib

# Function to fetch air pollution data
def fetch_air_pollution_data(lat, lon, start, end, api_key):
    url = "http://api.openweathermap.org/data/2.5/air_pollution/history"

    params = {
        "lat": lat,
        "lon": lon,
        "start": start,
        "end": end,
        "appid": api_key
    }
    response = requests.get(url, params=params)
    api_response = response.json()

    data = [
        {
            'datetime': datetime.utcfromtimestamp(item['dt']).strftime('%Y-%m-%d %H:%M:%S'),
            'aqi': item['main']['aqi'],
            **item['components']
        }
        for item in api_response['list']
    ]

    return pd.DataFrame(data)

# Function to fetch weather data
def fetch_monthly_data(lat, lon, start_date, end_date, api_key):
    base_url = "https://api.weatherbit.io/v2.0/history/hourly"
    params = {
        "lat": lat,
        "lon": lon,
        "start_date": start_date,
        "end_date": end_date,
        "key": api_key
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        return response.json().get('data', [])
    else:
        print(f"Error fetching data: {response.status_code}, {response.text}")
        return []

# Main function to execute the analysis
def main():
    # Hopsworks login
    #project_name = "AQIPredictions"
    #api_key = "b5GR3KdA1sCXrxzX.W29cSj9sMnypht8dLG4FYgjxebpKQxzbQPvOZPWpz5IyEko23UIXGiwAcuFENXfF"
    #connection = hopsworks.login(api_key=api_key)
    #project = connection.get_project(project_name)

    import hopsworks
    project = hopsworks.login()
    fs = project.get_feature_store(name='aqipredictions_featurestore')
    fg = fs.get_feature_group('aqipredictions', version=1)

    # Air pollution parameters
    api_key_pollution = "b219bcab2ed1426432de1ff7c768e46a"
    lat, lon = 33.44, -94.04
    start_timestamp = 1704067200  # Jan 1, 2024
    end_timestamp = 1735689600    # Dec 31, 2024

    # Fetch air pollution data
    data_air_pollution = fetch_air_pollution_data(lat, lon, start_timestamp, end_timestamp, api_key_pollution)
    data_air_pollution['datetime'] = pd.to_datetime(data_air_pollution['datetime'])
    data_air_pollution['day'] = data_air_pollution['datetime'].dt.day
    data_air_pollution['month'] = data_air_pollution['datetime'].dt.month
    data_air_pollution['date'] = data_air_pollution['datetime'].dt.date

    # Group by date and calculate the mean for numeric columns
    daily_data = data_air_pollution.groupby('date').mean().reset_index()
    daily_data['day'] = pd.to_datetime(daily_data['date']).dt.day
    daily_data['month'] = pd.to_datetime(daily_data['date']).dt.month

    # Weather data parameters
    api_key_weather = "a7475fbd2cb84f8ca148613df19e1ba4"
    start_date = datetime(2024, 1, 1)
    end_date = datetime(2024, 12, 31)

    # Fetch weather data
    current_date = start_date
    all_data = []
    while current_date < end_date:
        start_str = current_date.strftime('%Y-%m-%d')
        next_month = (current_date + timedelta(days=32)).replace(day=1)
        end_str = (next_month - timedelta(days=1)).strftime('%Y-%m-%d')

        print(f"Fetching data from {start_str} to {end_str}")
        monthly_data = fetch_monthly_data(lat, lon, start_str, end_str, api_key_weather)
        all_data.extend(monthly_data)

        current_date = next_month

    # Convert all data to a single DataFrame
    data_weather = pd.DataFrame(all_data)
    data_weather['datetime'] = data_weather['datetime'].str.replace(':', ' ')
    data_weather['datetime'] = pd.to_datetime(data_weather['datetime'])

    # Merge datasets
    merged_dataset = pd.merge(data_air_pollution, data_weather, on='datetime', how='inner')
    merged_dataset = merged_dataset.drop(columns=['h_angle', 'datetime', 'date', 'timestamp_local', 'timestamp_utc', 'pod', 'revision_status', 'weather'])

    # Prepare data for modeling
    X = merged_dataset.drop('aqi', axis=1)
    y = merged_dataset['aqi']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Create and train the model
    model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42)
    model.fit(X_train, y_train)

    # Evaluate the model
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"Mean Squared Error: {mse}")
    print(f"R2 Score: {r2}")

    # Predict the next three days
    future_dates = [data_air_pollution['datetime'].max() + timedelta(days=i) for i in range(1, 4)]
    future_data = pd.DataFrame(future_dates, columns=['datetime'])

    # Create features for future dates
    future_data['day'] = future_data['datetime'].dt.day
    future_data['month'] = future_data['datetime'].dt.month
    future_data['year'] = future_data['datetime'].dt.year
    future_data['day_of_week'] = future_data['datetime'].dt.dayofweek
    future_data['hour'] = 0  # Assuming we want to predict for the start of the day

    # Simulate or estimate values for missing features
    mean_values = X_train.mean()
    for col in mean_values.index:
        if col not in future_data.columns:
            future_data[col] = mean_values[col]

    # Ensure the future data has the same features as the training data
    future_data = future_data[X_train.columns]  # Align future data with training features

    # Make predictions for the next three days
    future_predictions = model.predict(future_data)

    # Display predictions
    for i, date in enumerate(future_dates):
        print(f"Predicted AQI for {date.date()}: {future_predictions[i]}")

  # Save the model
    model_path = "aqi_model.joblib"  # Save in the current directory
    joblib.dump(model, model_path)

    # Register the model in Hopsworks
    model_name = "AQI_Prediction_Model"
    model_version = "1.0"

    # Get the model registry
    model_registry = project.get_model_registry()

    # Check if the model already exists
    try:
        model = model_registry.get_model(model_name, model_version)
        print(f"Model {model_name} version {model_version} already exists.")
    except Exception as e:
        print(f"Model not found, registering new model: {model_name} version {model_version}")

    try:
            # Attempt to get the feature group
        feature_group = project.get_feature_group("aqipredictions")  # Adjust this if necessary
        feature_group.insert(data_air_pollution)
        print("Data inserted into feature group successfully.")
    except AttributeError as e:
        print("Error: The 'get_feature_group' method does not exist in the Project object.")
        print(e)
    except Exception as e:
        print(f"An error occurred while accessing the feature group: {e}")


    #feature_group = project.get_feature_group("aqipredictions")
    #feature_group.insert(data_air_pollution)

if __name__ == "__main__":
    main()

Connection closed.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1207481
Fetching data from 2024-01-01 to 2024-01-31
Fetching data from 2024-02-01 to 2024-02-29
Error fetching data: 429, {"status_code":429,"status_message":"Your request count (1504) is over the allowed limit of 1500 per day - Upgrade your key, or retry after 268.95 minutes"}

Fetching data from 2024-03-01 to 2024-03-31
Error fetching data: 429, {"status_code":429,"status_message":"Your request count (1534) is over the allowed limit of 1500 per day - Upgrade your key, or retry after 268.95 minutes"}

Fetching data from 2024-04-01 to 2024-04-30
Fetching data from 2024-05-01 to 2024-05-31
Error fetching data: 429, {"status_code":429,"status_message":"Your request count (1564) is over the allowed limit of 1500 per day - Upgrade your key, or retry after 268.91666666667 minutes"}

Fetching data from 2024-06-01 to 2024-06-30
Error fetching data: 429, {"status_code":429,"status_message":"Your request 

# New Section

In [10]:
pip show hopsworks

Name: hopsworks
Version: 4.1.5
Summary: Hopsworks Python SDK to interact with Hopsworks Platform, Feature Store, Model Registry and Model Serving
Home-page: https://www.hopsworks.ai
Author: 
Author-email: Hopsworks AB <robin@hopsworks.ai>
License: Apache-2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: avro, boto3, fsspec, furl, grpcio, hopsworks_aiomysql, mock, opensearch-py, pandas, protobuf, pyhumps, pyjks, PyMySQL, requests, retrying, sqlalchemy, tqdm, tzlocal
Required-by: 


In [ ]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

MODEL REGISTERED

In [35]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import hopsworks
import joblib
import openmeteo_requests
import requests_cache
from retry_requests import retry

# Function to fetch air pollution data
def fetch_air_pollution_data(lat, lon, start, end, api_key):
    url = "http://api.openweathermap.org/data/2.5/air_pollution/history"

    params = {
        "lat": lat,
        "lon": lon,
        "start": start,
        "end": end,
        "appid": api_key
    }
    response = requests.get(url, params=params)
    api_response = response.json()

    data = [
        {
            'datetime': datetime.utcfromtimestamp(item['dt']).strftime('%Y-%m-%d %H:%M:%S'),
            'aqi': item['main']['aqi'],
            **item['components']
        }
        for item in api_response['list']
    ]

    return pd.DataFrame(data)

# Function to fetch weather data using Open-Meteo API
def fetch_weather_data(lat, lon, start_date, end_date):
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
    retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
    openmeteo = openmeteo_requests.Client(session=retry_session)

    # Define the API parameters
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start_date,
        "end_date": end_date,
        "hourly": [
            "temperature_2m", "dew_point_2m", "precipitation", "rain",
            "snowfall", "weather_code", "pressure_msl", "surface_pressure",
            "cloud_cover", "vapour_pressure_deficit", "wind_speed_10m",
            "soil_temperature_0_to_7cm"
        ],
        "timezone": "Asia/Singapore"
    }

    # Fetch the weather data
    responses = openmeteo.weather_api(url, params=params)
    return responses[0]  # Return the first response

# Main function to execute the analysis
def main():
    # Hopsworks login
    project = hopsworks.login()
    fs = project.get_feature_store(name='aqipredictions_featurestore')
    fg = fs.get_feature_group('aqipredictions', version=1)

    # Air pollution parameters
    api_key_pollution = "b219bcab2ed1426432de1ff7c768e46a"
    lat, lon = 33.44, -94.04
    start_timestamp = 1704067200  # Jan 1, 2024
    end_timestamp = 1735689600    # Dec 31, 2024


    # Fetch air pollution data
    data_air_pollution = fetch_air_pollution_data(lat, lon, start_timestamp, end_timestamp, api_key_pollution)
    data_air_pollution['datetime'] = pd.to_datetime(data_air_pollution['datetime'])
    data_air_pollution['day'] = data_air_pollution['datetime'].dt.day
    data_air_pollution['month'] = data_air_pollution['datetime'].dt.month
    data_air_pollution['date'] = data_air_pollution['datetime'].dt.date

    # Group by date and calculate the mean for numeric columns
    daily_data = data_air_pollution.groupby('date').mean().reset_index()
    daily_data['day'] = pd.to_datetime(daily_data['date']).dt.day
    daily_data['month'] = pd.to_datetime(daily_data['date']).dt.month

    # Weather data parameters
    start_date = "2024-01-01"
    end_date = "2024-12-31"

    # Fetch weather data using Open-Meteo API
    weather_response = fetch_weather_data(lat, lon, start_date, end_date)

    # Process hourly data from Open-Meteo response
    hourly = weather_response.Hourly()
    hourly_data = {
        "date": pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        )
    }

    # Extract hourly variables
    hourly_data["temperature_2m"] = hourly.Variables(0).ValuesAsNumpy()
    hourly_data["dew_point_2m"] = hourly.Variables(1).ValuesAsNumpy()
    hourly_data["precipitation"] = hourly.Variables(2).ValuesAsNumpy()
    hourly_data["rain"] = hourly.Variables(3).ValuesAsNumpy()
    hourly_data["snowfall"] = hourly.Variables(4).ValuesAsNumpy()
    hourly_data["weather_code"] = hourly.Variables(5).ValuesAsNumpy()
    hourly_data["pressure_msl"] = hourly.Variables(6).ValuesAsNumpy()
    hourly_data["surface_pressure"] = hourly.Variables(7).ValuesAsNumpy()
    hourly_data["cloud_cover"] = hourly.Variables(8).ValuesAsNumpy()
    hourly_data["vapour_pressure_deficit"] = hourly.Variables(9).ValuesAsNumpy()
    hourly_data["wind_speed_10m"] = hourly.Variables(10).ValuesAsNumpy()
    hourly_data["soil_temperature_0_to_7cm"] = hourly.Variables(11).ValuesAsNumpy()

    # Convert all data to a single DataFrame
    data_weather = pd.DataFrame(data=hourly_data)
    data_weather['date'] = data_weather['date'].dt.date  # Ensure it's a date type

    # Merge datasets
    merged_dataset = pd.merge(data_air_pollution, data_weather, on='date', how='inner')

    # Prepare data for modeling
    X = merged_dataset.drop('aqi', axis=1)

    # Convert datetime columns to numerical features if they exist
    if 'datetime' in X.columns:
        X['day'] = X['datetime'].dt.day
        X['month'] = X['datetime'].dt.month
        X['year'] = X['datetime'].dt.year
        X.drop('datetime', axis=1, inplace=True)  # Drop the original datetime column

    # Ensure all columns are numeric
    X = X.select_dtypes(include=['number'])

    y = merged_dataset['aqi']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Create and train the model
    model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42)
    model.fit(X_train, y_train)

    # Evaluate the model
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"Mean Squared Error: {mse}")
    print(f"R2 Score: {r2}")

    # Predict the next three days
    future_dates = [data_air_pollution['datetime'].max() + timedelta(days=i) for i in range(1, 4)]
    future_data = pd.DataFrame(future_dates, columns=['datetime'])

    # Create features for future dates
    future_data['day'] = future_data['datetime'].dt.day
    future_data['month'] = future_data['datetime'].dt.month
    future_data['year'] = future_data['datetime'].dt.year
    future_data['day_of_week'] = future_data['datetime'].dt.dayofweek
    future_data['hour'] = 0  # Assuming we want to predict for the start of the day

    # Simulate or estimate values for missing features
    mean_values = X_train.mean()
    for col in mean_values.index:
        if col not in future_data.columns:
            future_data[col] = mean_values[col]

    # Ensure the future data has the same features as the training data
    future_data = future_data[X_train.columns]  # Align future data with training features

    # Make predictions for the next three days
    future_predictions = model.predict(future_data)

    # Display predictions
    for i, date in enumerate(future_dates):
        print(f"Predicted AQI for {date.date()}: {future_predictions[i]}")

    # Save the trained model locally
    model_path = 'aqi_model.joblib'  # Path to save the model
    joblib.dump(model, model_path)  # Save the model using joblib

    # Step 1: Log in to Hopsworks
    project = hopsworks.login()

    # Step 2: Get the Model Registry
    mr = project.get_model_registry()

    # Step 3: Define input and output schemas
    input_schema = Schema(X_train)  # Create schema for input features
    output_schema = Schema(y_train)  # Create schema for output (AQI values)
    model_schema = ModelSchema(input_schema, output_schema)  # Combine schemas

    # Step 4: Define metrics
    metrics = {
        "mean_squared_error": mse,  # Replace with your calculated MSE
        "r2_score": r2  # Replace with your calculated R2 score
    }

    # Step 5: Create and register the model in the model registry
    aqi_model = mr.python.create_model(
        version=1,
        name="AQI_Prediction_Model",  # Name of the model
        metrics=metrics,  # Metrics for the model
        model_schema=model_schema,  # Input and output schema
        input_example=X_train.sample(),  # Example input for validation
        description="Model for predicting Air Quality Index (AQI)"
    )

    # Step 6: Save the model to the registry
    aqi_model.save(model_path)  # Save the model file to the model registry

    print("Model registered successfully in Hopsworks Model Registry.")

if __name__ == "__main__":
    main()

Connection closed.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1207481
Mean Squared Error: 2.8957157115660703e-10
R2 Score: 0.9999999992944667
Predicted AQI for 2025-01-02: 1.0000160065488293
Predicted AQI for 2025-01-03: 1.0000160065488293
Predicted AQI for 2025-01-04: 1.0000160065488293
Connection closed.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1207481


  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/284252 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/233 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/1820 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/1207481/models/AQI_Prediction_Model/1
Model registered successfully in Hopsworks Model Registry.
